In [34]:
import pandas as pd
import regex
import numpy as np
import warnings
import random
import json
warnings.filterwarnings("ignore", category=Warning)


In [36]:
df2 = pd.read_csv(r"data/trialsdata/drugs.csv")
df2.sample(n=5)

,itemLabel,altLabel_list
1942,trimebutine,"2-Dimethylamino-2-phenylbutyl 3,4,5-trimethoxy..."
78,ethoxzolamide,"6-Ethoxy-1,3-benzothiazole-2-sulfonamide|6-eth..."
276,ketoprofen,Actron|2-(3-Benzoylphenyl)propionic acid|3-Ben...
2277,Alirocumab,REGN727|SAR-236553|SAR236553
454,difenoxin,"1-(3-cyano-3,3-diphenylpropyl)-4-phenylisonipe..."


In [37]:
df2['alternatives'] = df2['itemLabel'] + "|" + df2['altLabel_list'] 
df2['alternatives'] = df2['alternatives'].str.lower().str.split('|')
df2

,itemLabel,altLabel_list,alternatives
0,propylthiouracil,2-Mercapto-6-propyl-4-pyrimidone|2-Mercapto-6-...,"[propylthiouracil, 2-mercapto-6-propyl-4-pyrim..."
1,gemfibrozil,"2,2-Dimethyl-5-(2,5-dimethylphenoxy)valeriansa...","[gemfibrozil, 2,2-dimethyl-5-(2,5-dimethylphen..."
2,isoflurophate,DFP|Diisopropoxyphosphoryl Fluoride|Diisopropy...,"[isoflurophate, dfp, diisopropoxyphosphoryl fl..."
3,6-thioguanine,"TG|2-Amino 6MP|2-Amino-1,7-dihydro-6H-purine-6...","[6-thioguanine, tg, 2-amino 6mp, 2-amino-1,7-d..."
4,melatonin,{N-[2-(5-methoxy-1H-indol-3-yl)ethyl]-} Acetam...,"[melatonin, {n-[2-(5-methoxy-1h-indol-3-yl)eth..."
...,...,...,...
2381,Somatrem,Protropin|Somatrem (genetical recombination),"[somatrem, protropin, somatrem (genetical reco..."
2382,spiraprilat,SCH-33861,"[spiraprilat, sch-33861]"
2383,stepronin,Stepronin|Stepronine|Stepronino|Steproninum|Ti...,"[stepronin, stepronin, stepronine, stepronino,..."
2384,sulfacytine,1-ethyl N4-sulfanilylcytosin|1-ethyl-N-sulfani...,"[sulfacytine, 1-ethyl n4-sulfanilylcytosin, 1-..."


In [55]:
df3 = pd.read_json(r"data/trialsdata/clinical_trials_2015.jsonl", lines = True)
df3_drugs = df3[df3['intervention_type']== "Drug"]
"""I filtered by intervention type drug here I was not 
expecting to find matches in the other categories. 
This can easily be changed with the following olumn if required."""
#df3_drugs = df3
df3_drugs['intervention_name'] = df3_drugs['intervention_name'].str.lower()
df3_drugs.sample(n=10)

,nct_id,intervention_name,intervention_type
13115,NCT02507440,propofol,Drug
13207,NCT02509299,standard medical treatment,Drug
4843,NCT02366585,ciclosporin,Drug
17362,NCT02581761,cytotec,Drug
24566,NCT02946606,gx-h9,Drug
23515,NCT02831894,10% hypnotic medication taper,Drug
6230,NCT02393157,etoposide,Drug
523,NCT02037126,diphenhydramine,Drug
11640,NCT02483247,doxorubicin,Drug
22300,NCT02738177,misoprostol,Drug


In [65]:
words_to_remove = ["and", "intravenous", "+", "- ", "/", r"\(.*\)", "®", " or"]
df3_drugs['matches'] = df3_drugs['intervention_name']

for item in words_to_remove:
    df3_drugs['matches'] = df3_drugs['matches'].str.replace(item,",")

df3_drugs['matches'] = df3_drugs['matches'].str.lower().str.split(',')
#df3_drugs['matches']  = df3_drugs['matches'].str.replace(r"\(.*\)","")
df3_drugs.sample(n=5)#.intervention_type.sample(n=1).reset_index()

,nct_id,intervention_name,intervention_type,matches
17506,NCT02584582,liraglutide,Drug,[liraglutide]
16907,NCT02573831,oxycodone,Drug,[oxycodone]
19055,NCT02613806,normal saline,Drug,[normal saline]
10173,NCT02459392,depomedrol,Drug,[depomedrol]
13467,NCT02513355,taxol,Drug,[taxol]


In [68]:
df3_drugs.sample(n=15)

,nct_id,intervention_name,intervention_type,matches
13292,NCT02510612,placebo,Drug,[placebo]
18272,NCT02598635,cholecalciferol,Drug,[cholecalciferol]
24016,NCT02882984,gefitinib 250mg po qd or tarceva 150mg po qd o...,Drug,"[gefitinib 250mg po qd, tarceva 150mg po qd, ..."
6259,NCT02393573,metformin,Drug,[metformin]
23161,NCT02802111,albuterol,Drug,[albuterol]
19625,NCT02626572,s47445 5mg,Drug,[s47445 5mg]
3006,NCT02326727,fentanyl,Drug,[fentanyl]
11893,NCT02487082,melatonin and donepezil,Drug,"[melatonin , donepezil]"
4378,NCT02357303,acetylcysteine,Drug,[acetylcysteine]
22880,NCT02778789,tocilizumab,Drug,[tocilizumab]


In [78]:
drugs_list = list(df2['alternatives'])
#print(drugs_list)

def match_drugs(matches):
    """iterates over the full list of drugs available in drugs.csv, and
    cleaned using regex and string replace methods. Will return the first
    value of the list, as this is the value from 'itemLabel' in drugs.csv """
    
    #print(drugs_list)
    match = ""
    for item in matches:
        item = item.strip()
        for sublist in drugs_list:
            if item not in sublist:
                continue
            else:
                match += sublist[0]
                continue
    return match
            
             
df3_drugs['drugs'] = df3_drugs['matches'].apply(match_drugs)
df3_drugs
#can see on index 25644 that hyaluronan was matched to hyaluronic acid

,nct_id,intervention_name,intervention_type,matches,drugs
0,NCT00293735,labetalol (seizure prevention),Drug,"[labetalol , ]",labetalololmesartan
1,NCT00293735,mgso4 (seizure prevention),Drug,"[mgso4 , ]",magnesium sulfateolmesartan
9,NCT00749853,follicle stimulating hormone,Drug,[follicle stimulating hormone],urofollitropin
25,NCT01112878,clonidine,Drug,[clonidine],clonidine
26,NCT01112878,gabapentin,Drug,[gabapentin],gabapentin
...,...,...,...,...,...
25631,NCT03692299,metronidazole,Drug,[metronidazole],metronidazole
25638,NCT03705962,tobramycin,Drug,[tobramycin],tobramycin
25644,NCT03800810,hyaluronic acid,Drug,[hyaluronic acid],hyaluronan
25649,NCT03825939,intravenous placebo,Drug,"[, placebo]",olmesartan


In [82]:
df3_drugs = df3_drugs[df3_drugs['drugs'] != ""]
df3_drugs[df3_drugs['nct_id'] == 'NCT01969578']

,nct_id,intervention_name,intervention_type,matches,drugs


In [76]:
df_output = df3_drugs[['nct_id', 'drugs']]
df_output
#df_output = df_output.to_json()


,nct_id,drugs
0,NCT00293735,labetalololmesartan
1,NCT00293735,magnesium sulfateolmesartan
9,NCT00749853,urofollitropin
25,NCT01112878,clonidine
26,NCT01112878,gabapentin
...,...,...
25631,NCT03692299,metronidazole
25638,NCT03705962,tobramycin
25644,NCT03800810,hyaluronan
25649,NCT03825939,olmesartan


In [44]:
json_output = df_output.to_json(orient="records")

parsed = json.loads(json_output)
json.dumps(parsed, indent=4) 
#with open("task1b.json", "w") as outfile: 
#    json.dump(json_output, outfile)

'[\n    {\n        "nct_id": "NCT01112878",\n        "drugs": "clonidine"\n    },\n    {\n        "nct_id": "NCT01112878",\n        "drugs": "gabapentin"\n    },\n    {\n        "nct_id": "NCT01132482",\n        "drugs": "sildenafil"\n    },\n    {\n        "nct_id": "NCT01171482",\n        "drugs": "mitomycin"\n    },\n    {\n        "nct_id": "NCT01232647",\n        "drugs": "(e)-phytonadione"\n    },\n    {\n        "nct_id": "NCT01255046",\n        "drugs": "donepezilolmesartan"\n    },\n    {\n        "nct_id": "NCT01360034",\n        "drugs": "indomethacin"\n    },\n    {\n        "nct_id": "NCT01360034",\n        "drugs": "nifedipine"\n    },\n    {\n        "nct_id": "NCT01408849",\n        "drugs": "omeprazole"\n    },\n    {\n        "nct_id": "NCT01485393",\n        "drugs": "dexmedetomidine"\n    },\n    {\n        "nct_id": "NCT01485393",\n        "drugs": "zolpidem"\n    },\n    {\n        "nct_id": "NCT01528865",\n        "drugs": "lamivudine"\n    },\n    {\n        "nc

In [27]:
output_dict_nct = dict(zip(df_output.nct_id, df_output.drugs))
with open("task1.json", "w") as outfile: 
    json.dump(output_dict_nct, outfile)
output_dict_nct

{'NCT01112878': 'gabapentin',
 'NCT01132482': 'sildenafil',
 'NCT01171482': 'mitomycin',
 'NCT01232647': '(e)-phytonadione',
 'NCT01255046': 'donepezilolmesartan',
 'NCT01360034': 'nifedipine',
 'NCT01408849': 'omeprazole',
 'NCT01485393': 'zolpidem',
 'NCT01528865': 'lamivudine',
 'NCT01530984': 'ipilimumab',
 'NCT01561222': 'calcitriol',
 'NCT01589497': 'rifampicin',
 'NCT01613677': 'buparlisib',
 'NCT01614197': 'temsirolimus',
 'NCT01663558': 'imiquimod',
 'NCT01718444': 'progesterone',
 'NCT01733043': 'dexmedetomidine',
 'NCT01769911': 'melphalan',
 'NCT01824264': 'sitagliptin',
 'NCT01836614': 'lidocaine',
 'NCT01854762': 'raltegravir',
 'NCT01883297': 'cyclophosphamide',
 'NCT01885364': 'remifentanil',
 'NCT01890200': 'ribavirin',
 'NCT01922141': 'ramipril',
 'NCT01931098': 'topotecan',
 'NCT01933269': 'fluciclovine-f18',
 'NCT01940536': 'tranexamic acid',
 'NCT01942122': 'mefenamic acid',
 'NCT01953406': 'mitomycin',
 'NCT01954290': 'mannitol',
 'NCT01960075': '(s)-etiracetam',


# Task 2:
    

This matches the drugs found in the trials to the descriptions from usan_stems

In [11]:
df_usan_all = pd.read_csv(r"data/trialsdata/usan_stems.csv", header=None, sep='\n')
df_usan_all = df_usan_all[0].str.split(',', expand=True)
headers = list(df_usan_all.iloc[0].fillna("rename")) 
headers = [i.replace('\"', "") for i in headers]
print(headers)
#temporary renaming empty the empty columns
df_usan_all.columns = ['name', 'stem', 'definition', 'example', '1', '2', '3', '4', '5', '6']
df_usan_all.drop(index=0, inplace=True)
df_usan_all

['name', 'stem', 'definition', 'example', '', 'rename', 'rename', 'rename', 'rename', 'rename']


,name,stem,definition,example,1,2,3,4,5,6
1,abine,"""-abine""","""nucleoside antiviral or antineoplastic agents","cytarabine or azarabine derivatives""",,,None,None,None,None
2,ac,"""-ac""","""anti-inflammatory agents (acetic acid derivat...","""bromfenac""",,None,None,None,None,None
3,subgroup:,,,,,None,None,None,None,None
4,,"""-zolac""","""anti-inflammatory - pyrazole acetic acid deri...",""" pyrazole acetic acid derivatives""","""rovazolac""",None,None,None,None,None
5,acetam,"""-acetam""","""nootropic agents (learning",cognitive enhancers),"piracetam type""",,,None,None,None
...,...,...,...,...,...,...,...,...,...,...
758,zolid,"""-zolid""","""oxazolidinone antibacterials""","""eperezolid""",,None,None,None,None,None
759,zomib,"""-zomib""","""proteozome inhibitors""","""bortezomib""",,None,None,None,None,None
760,zotan,"""-zotan""","""5-HT1A receptor agonists/antagonists acting p...","""robalzotan""",,None,None,None,None,None
761,zotide,"""-zotide""","""peptides""",,,None,None,None,None,None


In [12]:

df_usan = df_usan_all.loc[~df_usan_all['name'].str.contains("subgroup:",regex=False)]
df_usan = df_usan.loc[~df_usan['name'].str.contains("subgroups:",regex=False)]
df_usan['name'] = df_usan['name'].replace("", np.NaN )
df_usan['name'] = df_usan['name'].fillna(method='ffill')
df_usan['definition'] = df_usan['definition'].str.strip(' " " ')
df_usan = df_usan.fillna("")
df_usan['example_all'] = df_usan['definition'] + "," + df_usan['example'] + ","  + df_usan['1'] + "," + df_usan['2'] + "," + df_usan['3'] + "," + df_usan['4'] + "," + df_usan['5'] + "," + df_usan['6'] 
df_usan['example_all'] = df_usan['example_all'].replace('\"', '')
df_usan.drop(columns = ['example', '1', '2', '3', '4', '5', '6'], inplace = True)
df_usan.sample(n=10)

,name,stem,definition,example_all
375,minib,"""-minib""",kinase inhibitor,"kinase inhibitor, binds to myristoyl binding s..."
518,punil,"""-punil""",mitochondrial benzodiazepine receptor (MBR) se...,mitochondrial benzodiazepine receptor (MBR) se...
92,butazone,"""-butazone""",anti-inflammatory analgesics (phenylbutazone t...,anti-inflammatory analgesics (phenylbutazone t...
678,tiostat,"""-tiostat""",enzyme inhibitors,"enzyme inhibitors,,,,,,,"
624,tant,"""-tant""",tachykinin (neurokinin) receptor antagonists,"tachykinin (neurokinin) receptor antagonists,,..."
91,butan,"""-butan""",antiseptics (dapabutan type),"antiseptics (dapabutan type),""lopobutan"",,,,,,"
724,vir,"""-atovir""",respiratory syncytial virus (RSV) fusion inhib...,respiratory syncytial virus (RSV) fusion inhib...
601,stat,"""-tristat""",tryptophan hydroxylase inhibitors,"tryptophan hydroxylase inhibitors,""telotristat..."
279,gramostim,"""-gramostim""",colony-stimulating factors,"colony-stimulating factors,,,,,,,"
303,irudin,"""-irudin""",anticoagulants (hirudin type),"anticoagulants (hirudin type),""desirudin"",,,,,,"


In [87]:
#dict(df_usan[['name', 'definition']])
usan_dict = dict(zip(df_usan.name, df_usan.definition))
#print(usan_dict)
output_list_usan = []

for drug in list(df3_drugs['drugs']):
    #print("the drug: ", drug)
    drug_usan = {}
    usan_codes = []
    drug_usan['drug'] = drug
    for k, v in usan_dict.items(): 
        if drug.endswith(k): 
            usandict = {}
            #print("k ", k)
            usandict['description'] = v
            usan_codes.append(usandict)
            continue
            #print(usan_codes)
    if len(usan_codes) > 0:
        drug_usan['usan_codes'] = usan_codes
        output_list_usan.append(drug_usan)
            
#drugs_usan    
#print(list(df3_drugs['drugs']))
output_list_usan

[{'drug': 'labetalololmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': 'magnesium sulfateolmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': 'sildenafil',
  'usan_codes': [{'description': '(PDE5) inhibitors containing a sulfonamide moiety'}]},
 {'drug': 'fluorouracil',
  'usan_codes': [{'description': 'uracil type antineoplastics'},
   {'description': 'uracil derivatives used as thyroid antagonists and as antineoplastics'}]},
 {'drug': 'mitomycin',
  'usan_codes': [{'description': 'antibiotics (Streptomyces strain)'}]},
 {'drug': 'sorafenib',
  'usan_codes': [{'description': 'rapidly accelerated fibrosarcoma (RAF) kinase inhibitors'}]},
 {'drug': 'olmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': 'donepezilolmesartan',
  'usan_codes': [{'description': 'angiotensin II receptor antagonists'}]},
 {'drug': 'preotactolmesartan',
  'usan_codes': [{'desc

# Task 3:

In [88]:
def task_3(output_list_usan, output_dict_nct):
    output_list_3 = []
    for drug in output_list_usan:
        drug_description = {}
        trials = []
        #print(drug['drug'])
        #print(drug["usan_codes"][0]["description"])
        for k,v in output_dict_nct.items():
            #print(k, v)
            if v == drug['drug']:
                #print("YES")                
                trials.append(k)        
        if len(trials) > 0:
            drug_description['description'] = drug["usan_codes"][0]["description"]
            drug_description['trials'] = trials
            output_list_3.append(drug_description)
    return output_list_3
                

In [89]:
task_3(output_list_usan, output_dict_nct)

[{'description': '(PDE5) inhibitors containing a sulfonamide moiety',
  'trials': ['NCT01132482',
   'NCT02277132',
   'NCT02304705',
   'NCT02337881',
   'NCT02364882',
   'NCT02387450',
   'NCT02414204',
   'NCT02524184',
   'NCT02595541',
   'NCT02626182',
   'NCT02795351']},
 {'description': 'uracil type antineoplastics',
  'trials': ['NCT02235324', 'NCT02325986', 'NCT02500940', 'NCT02573220']},
 {'description': 'antibiotics (Streptomyces strain)',
  'trials': ['NCT01171482', 'NCT01953406', 'NCT02106572']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT02407210',
   'NCT02525822',
   'NCT02547038',
   'NCT02619799',
   'NCT02646332',
   'NCT02996916']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01255046']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT02407210',
   'NCT02525822',
   'NCT02547038',
   'NCT02619799',
   'NCT02646332',
   'NCT02996916']},
 {'description': '(nifedipine type)',
  'tria

# Task 4:

In [91]:
task3_list = task_3(output_list_usan, output_dict_nct)
task3_list

[{'description': '(PDE5) inhibitors containing a sulfonamide moiety',
  'trials': ['NCT01132482',
   'NCT02277132',
   'NCT02304705',
   'NCT02337881',
   'NCT02364882',
   'NCT02387450',
   'NCT02414204',
   'NCT02524184',
   'NCT02595541',
   'NCT02626182',
   'NCT02795351']},
 {'description': 'uracil type antineoplastics',
  'trials': ['NCT02235324', 'NCT02325986', 'NCT02500940', 'NCT02573220']},
 {'description': 'antibiotics (Streptomyces strain)',
  'trials': ['NCT01171482', 'NCT01953406', 'NCT02106572']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT02407210',
   'NCT02525822',
   'NCT02547038',
   'NCT02619799',
   'NCT02646332',
   'NCT02996916']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT01255046']},
 {'description': 'angiotensin II receptor antagonists',
  'trials': ['NCT02407210',
   'NCT02525822',
   'NCT02547038',
   'NCT02619799',
   'NCT02646332',
   'NCT02996916']},
 {'description': '(nifedipine type)',
  'tria

In [98]:
def make_task4_list():
    trial_counts_list= []

    for i in range(0,len(task3_list)-1,2):
        trial_counts_dict = {}
        trial_counts_dict["description1"] = task3_list[i]["description"]
        trial_counts_dict["description2"] = task3_list[i+1]["description"]
        trial_counts_dict["trial_count"] = len(task3_list[i]["trials"]) + len(task3_list[i+1]["trials"])
        print(trial_counts_dict)
        trial_counts_list.append(trial_counts_dict)
        return trial_counts_list
    
trial_counts_list
    

{'description1': '(PDE5) inhibitors containing a sulfonamide moiety', 'description2': 'uracil type antineoplastics', 'trial_count': 15}
{'description1': 'antibiotics (Streptomyces strain)', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 9}
{'description1': 'angiotensin II receptor antagonists', 'description2': 'angiotensin II receptor antagonists', 'trial_count': 7}
{'description1': '(nifedipine type)', 'description2': 'proton pump inhibitors', 'trial_count': 13}
{'description1': 'hypnotics/sedatives (zolpidem type)', 'description2': 'antineoplastics', 'trial_count': 9}
{'description1': 'monoclonal antibodies', 'description2': 'antibacterials (quinolone derivatives)', 'trial_count': 12}
{'description1': 'phosphatidylinositol 3-kinase (PI3K) inhibitors', 'description2': 'nucleoside antiviral or antineoplastic agents', 'trial_count': 7}
{'description1': 'antimetabolites (folic acid derivatives)', 'description2': 'immunosuppressant', 'trial_count': 14}
{'description

[{'description1': '(PDE5) inhibitors containing a sulfonamide moiety',
  'description2': 'uracil type antineoplastics',
  'trial_count': 15},
 {'description1': 'antibiotics (Streptomyces strain)',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 9},
 {'description1': 'angiotensin II receptor antagonists',
  'description2': 'angiotensin II receptor antagonists',
  'trial_count': 7},
 {'description1': '(nifedipine type)',
  'description2': 'proton pump inhibitors',
  'trial_count': 13},
 {'description1': 'hypnotics/sedatives (zolpidem type)',
  'description2': 'antineoplastics',
  'trial_count': 9},
 {'description1': 'monoclonal antibodies',
  'description2': 'antibacterials (quinolone derivatives)',
  'trial_count': 12},
 {'description1': 'phosphatidylinositol 3-kinase (PI3K) inhibitors',
  'description2': 'nucleoside antiviral or antineoplastic agents',
  'trial_count': 7},
 {'description1': 'antimetabolites (folic acid derivatives)',
  'description2': 'immunos